In [1]:
import os
import numpy as np
from scipy.stats import zscore
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import seaborn as sns
import brainiak.eventseg.event

In [ ]:

segment_data_dir = '/Volumes/ARCHIVES/thesis_pipeline/data/hmm_segment_data_fixed'
alignment_dir = '/Volumes/ARCHIVES/thesis_pipeline/data/alignment_matrices_fixed'


os.makedirs(alignment_dir, exist_ok=True)

# Function to compute z-scored alignment between two boundary sets
def match_z(proposed_bounds, gt_bounds, num_TRs):
    nPerm = 1000  # Number of permutations
    threshold = 3  # Tolerance for alignment
    np.random.seed(0)

    gt_lengths = np.diff(np.concatenate(([0], gt_bounds, [num_TRs])))
    match = np.zeros(nPerm + 1)
    for p in range(nPerm + 1):
        gt_bounds = np.cumsum(gt_lengths)[:-1]
        for b in gt_bounds:
            if np.any(np.abs(proposed_bounds - b) <= threshold):
                match[p] += 1
        match[p] /= len(gt_bounds)
        gt_lengths = np.random.permutation(gt_lengths)  # Shuffle boundary lengths for permutation test
    
    # Return the z-score for alignment
    return (match[0] - np.mean(match[1:])) / np.std(match[1:])

# Load event boundaries for each subject and region
def load_event_boundaries(roi, n_subjects):
    boundaries = []
    for subj_idx in range(n_subjects):
        segment_file = os.path.join(segment_data_dir, f'{roi}_subject_{subj_idx + 1}_segments.npy')
        if os.path.exists(segment_file):
            bounds = np.load(segment_file, allow_pickle=True)
            boundaries.append(bounds)
        else:
            print(f"Event boundary data not found for {roi}, subject {subj_idx + 1}")
    return boundaries

# Function to compute the subject-by-subject alignment matrix for a given ROI
def compute_alignment_matrix(roi, n_subjects, n_TRs):
    boundaries = load_event_boundaries(roi, n_subjects)
    
    # Initialize the subject-by-subject alignment matrix
    matchz_mat = np.zeros((n_subjects, n_subjects))
    
    for i in range(n_subjects):
        for j in range(n_subjects):
            if i != j:
                matchz_mat[i, j] = match_z(boundaries[i], boundaries[j], n_TRs)
    
    # Save the alignment matrix in the new directory
    save_path = os.path.join(alignment_dir, f"matchz_{roi}_alignment.npy")
    np.save(save_path, matchz_mat)
    print(f"Saved alignment matrix for {roi} at {save_path}")

# Main function to compute alignment for all regions and subjects
def run_alignment_analysis(roi_names, n_subjects, n_TRs):
    Parallel(n_jobs=30, require='sharedmem')(
        delayed(compute_alignment_matrix)(roi, n_subjects, n_TRs) for roi in roi_names
    )


roi_names = ['PTL', 'ATL', 'AG', 'IFG', 'MFG', 'IFGorb'] 
n_subjects = 8  
n_TRs = 1291  


run_alignment_analysis(roi_names, n_subjects, n_TRs)

Saved alignment matrix for IFG at /Volumes/ARCHIVES/thesis_pipeline/data/alignment_matrices_fixed/matchz_IFG_alignment.npy
Saved alignment matrix for PTL at /Volumes/ARCHIVES/thesis_pipeline/data/alignment_matrices_fixed/matchz_PTL_alignment.npy
Saved alignment matrix for AG at /Volumes/ARCHIVES/thesis_pipeline/data/alignment_matrices_fixed/matchz_AG_alignment.npy
Saved alignment matrix for MFG at /Volumes/ARCHIVES/thesis_pipeline/data/alignment_matrices_fixed/matchz_MFG_alignment.npy
Saved alignment matrix for ATL at /Volumes/ARCHIVES/thesis_pipeline/data/alignment_matrices_fixed/matchz_ATL_alignment.npySaved alignment matrix for IFGorb at /Volumes/ARCHIVES/thesis_pipeline/data/alignment_matrices_fixed/matchz_IFGorb_alignment.npy



In [9]:


alignment_dir = '/Volumes/ARCHIVES/thesis_pipeline/data/alignment_matrices_fixed'
save_dir = '/Volumes/ARCHIVES/thesis_pipeline/figures/z_allignment_fixed'


os.makedirs(save_dir, exist_ok=True)

# Function to visualize and save IS-RSA matrix for each region
def visualize_and_save_is_rsa_matrix(roi_names, n_subjects):
    for roi in roi_names:
        # Path to the alignment matrix file for the specified region
        matrix_file = os.path.join(alignment_dir, f"matchz_{roi}_alignment.npy")
        
        if os.path.exists(matrix_file):
            # Load the alignment matrix
            alignment_matrix = np.load(matrix_file)
            
            # Plot the alignment matrix as a heatmap
            plt.figure(figsize=(10, 8))
            sns.heatmap(alignment_matrix, annot=True, cmap='coolwarm', 
                        xticklabels=range(1, n_subjects + 1), yticklabels=range(1, n_subjects + 1))
            plt.title(f'IS-RSA Matrix for {roi} (Subject-by-Subject)')
            plt.xlabel('Subjects')
            plt.ylabel('Subjects')
            
            # Save the figure
            save_path = os.path.join(save_dir, f'{roi}_IS_RSA.png')
            plt.savefig(save_path, dpi=300)
            plt.close()  # Close the figure to free up memory
            
            print(f"Saved IS-RSA matrix plot for {roi} at {save_path}")
        else:
            print(f"Alignment matrix for {roi} not found at {matrix_file}")


roi_names = ['PTL', 'ATL', 'AG', 'IFG', 'MFG', 'IFGorb']  # Regions to plot
n_subjects = 8  # Replace with the actual number of subjects


visualize_and_save_is_rsa_matrix(roi_names, n_subjects)


Saved IS-RSA matrix plot for PTL at /Volumes/ARCHIVES/thesis_pipeline/figures/z_allignment_fixed/PTL_IS_RSA.png
Saved IS-RSA matrix plot for ATL at /Volumes/ARCHIVES/thesis_pipeline/figures/z_allignment_fixed/ATL_IS_RSA.png
Saved IS-RSA matrix plot for AG at /Volumes/ARCHIVES/thesis_pipeline/figures/z_allignment_fixed/AG_IS_RSA.png
Saved IS-RSA matrix plot for IFG at /Volumes/ARCHIVES/thesis_pipeline/figures/z_allignment_fixed/IFG_IS_RSA.png
Saved IS-RSA matrix plot for MFG at /Volumes/ARCHIVES/thesis_pipeline/figures/z_allignment_fixed/MFG_IS_RSA.png
Saved IS-RSA matrix plot for IFGorb at /Volumes/ARCHIVES/thesis_pipeline/figures/z_allignment_fixed/IFGorb_IS_RSA.png


In [3]:
import os
import numpy as np
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import seaborn as sns

# Path to segmentation data and alignment matrices
segment_data_dir = '/Volumes/ARCHIVES/thesis_pipeline/data/hmm_segment_data_fixed'
alignment_dir = '/Volumes/ARCHIVES/thesis_pipeline/data/alignment_matrices_fixed_ROI'
figure_dir = '/Volumes/ARCHIVES/thesis_pipeline/data/alignment_matrices_fixed_ROI/figures'

# Ensure the directories exist
os.makedirs(alignment_dir, exist_ok=True)
os.makedirs(figure_dir, exist_ok=True)

# Function to compute z-scored alignment between two boundary sets
def match_z(proposed_bounds, gt_bounds, num_TRs):
    nPerm = 1000  # Number of permutations
    threshold = 3  # Tolerance for alignment
    np.random.seed(0)

    gt_lengths = np.diff(np.concatenate(([0], gt_bounds, [num_TRs])))
    match = np.zeros(nPerm + 1)
    for p in range(nPerm + 1):
        gt_bounds = np.cumsum(gt_lengths)[:-1]
        for b in gt_bounds:
            if np.any(np.abs(proposed_bounds - b) <= threshold):
                match[p] += 1
        match[p] /= len(gt_bounds)
        gt_lengths = np.random.permutation(gt_lengths)  # Shuffle boundary lengths for permutation test
    
    # Return the z-score for alignment
    return (match[0] - np.mean(match[1:])) / np.std(match[1:])

# Function to load event boundaries for a single subject across all regions
def load_event_boundaries_for_subject(roi_names, subject_idx):
    boundaries = {}
    for roi in roi_names:
        segment_file = os.path.join(segment_data_dir, f'{roi}_subject_{subject_idx + 1}_segments.npy')
        if os.path.exists(segment_file):
            boundaries[roi] = np.load(segment_file, allow_pickle=True)
        else:
            print(f"Event boundary data not found for {roi}, subject {subject_idx + 1}")
    return boundaries

# Function to compute region-by-region alignment matrix for a given subject
def compute_region_alignment_matrix(boundaries, roi_names, n_TRs):
    n_rois = len(roi_names)
    matchz_mat = np.zeros((n_rois, n_rois))

    # Compute z-score alignment between each region pair for the subject
    for i, roi1 in enumerate(roi_names):
        for j, roi2 in enumerate(roi_names):
            if i != j:
                matchz_mat[i, j] = match_z(boundaries[roi1], boundaries[roi2], n_TRs)
    
    return matchz_mat

# Plot and save alignment matrix as a heatmap
def plot_alignment_matrix(matrix, roi_names, subject_idx):
    plt.figure(figsize=(10, 8))
    sns.heatmap(matrix, annot=True, cmap='coolwarm', xticklabels=roi_names, yticklabels=roi_names)
    plt.title(f'Region Alignment Matrix (Within-Subject) - Subject {subject_idx + 1}')
    plt.xlabel('Regions')
    plt.ylabel('Regions')
    
  
    save_path = os.path.join(figure_dir, f'subject_{subject_idx + 1}_region_alignment_zscore.png')
    plt.savefig(save_path, dpi=300)
    plt.close()  # Close the figure to free up memory
    print(f"Saved alignment matrix for Subject {subject_idx + 1} at {save_path}")

# Main function to compute within-subject between-region alignment
def run_within_subject_region_alignment(roi_names, n_subjects, n_TRs):
    for subject_idx in range(n_subjects):
        # Load event boundaries for this subject across all regions
        boundaries = load_event_boundaries_for_subject(roi_names, subject_idx)

        # Compute the region-by-region alignment matrix for this subject
        alignment_matrix = compute_region_alignment_matrix(boundaries, roi_names, n_TRs)

        # Save the alignment matrix
        alignment_save_path = os.path.join(alignment_dir, f'subject_{subject_idx + 1}_region_alignment.npy')
        np.save(alignment_save_path, alignment_matrix)
        print(f"Saved alignment matrix for Subject {subject_idx + 1} at {alignment_save_path}")

        # Plot and save the alignment matrix figure
        plot_alignment_matrix(alignment_matrix, roi_names, subject_idx)

In [4]:
# Example usage for all regions and subjects
roi_names = ['PTL', 'ATL', 'AG', 'IFG', 'MFG', 'IFGorb']  # List of regions
n_subjects = 8  # Number of subjects
n_TRs = 1291  # Total number of TRs

# Run the within-subject between-region alignment analysis
run_within_subject_region_alignment(roi_names, n_subjects, n_TRs)

Saved alignment matrix for Subject 1 at /Volumes/ARCHIVES/thesis_pipeline/data/alignment_matrices_fixed_ROI/subject_1_region_alignment.npy
Saved alignment matrix for Subject 1 at /Volumes/ARCHIVES/thesis_pipeline/data/alignment_matrices_fixed_ROI/figures/subject_1_region_alignment_zscore.png
Saved alignment matrix for Subject 2 at /Volumes/ARCHIVES/thesis_pipeline/data/alignment_matrices_fixed_ROI/subject_2_region_alignment.npy
Saved alignment matrix for Subject 2 at /Volumes/ARCHIVES/thesis_pipeline/data/alignment_matrices_fixed_ROI/figures/subject_2_region_alignment_zscore.png
Saved alignment matrix for Subject 3 at /Volumes/ARCHIVES/thesis_pipeline/data/alignment_matrices_fixed_ROI/subject_3_region_alignment.npy
Saved alignment matrix for Subject 3 at /Volumes/ARCHIVES/thesis_pipeline/data/alignment_matrices_fixed_ROI/figures/subject_3_region_alignment_zscore.png
Saved alignment matrix for Subject 4 at /Volumes/ARCHIVES/thesis_pipeline/data/alignment_matrices_fixed_ROI/subject_4_reg